<a href="https://colab.research.google.com/github/bz2310/final_project/blob/main/final_project_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import numpy as np
from google.colab import drive
drive.mount("drive", force_remount=True)

Mounted at drive


In [2]:
root_path = '/content/drive/MyDrive/Colab Notebooks/final_project'

# Collect Base Model Results

In [171]:
def process_name(final_results):
  for key in ['model', 'window', 'batchsize', 'layers', 'l2', 'dropout', 'lstmsize']:
    try:
      final_results[key] = final_results.reset_index()['name'].apply(lambda x: [y for y in x.split("_") if key + "." in y][0].split(".")[1]).values
    except:
      pass
  return final_results

def collect_results(filenames):
  import pickle
  final_results = pd.DataFrame()
  for file in filenames:
    res = pickle.load(open(file, 'rb'))
    res['name'] = file.split("/")[-1]
    res['timing'] = pd.read_csv(file.replace(".pickle", "_timingcb.csv"), index_col=0)['0'].values
    final_results = pd.concat([final_results, pd.DataFrame(res).reset_index().rename(columns={'index':'epoch'})], axis=0)
  return final_results

In [ ]:
import glob
filenames = glob.glob(root_path + "/*seq_*batchsize.32*.pickle")
filenames.sort()
final_results = collect_results(filenames)

In [158]:
## get results from final epoch
final_results = final_results.groupby('name').apply(lambda x: x.iloc[-1]).sort_values(by='test_eval').drop('name', axis=1)
final_results = process_name(final_results)

In [190]:
final_results.drop(['timing', 'epoch'], axis=1).groupby(['model', 'window', 'batchsize', 'layers']).mean().style.background_gradient().set_precision(3)

# Collect LSTM Results

In [172]:
import glob
filenames = glob.glob(root_path + "/*lstm_*batchsize.32*.pickle")
filenames.sort()
lstm_results = collect_results(filenames)

In [173]:
lstm_results = process_name(lstm_results)

In [194]:
lstm_results[lstm_results.window=='ts'].assign(lstmsize=lambda df: df.lstmsize.astype(np.int)).groupby(['lstmsize']).last()[['loss', 'val_loss', 'test_eval', 'timing']].style.background_gradient().set_precision(4)

,loss,val_loss,test_eval,timing
lstmsize,,,,
8,0.0059,0.0059,0.0059,15.6194
16,0.0056,0.0055,0.0055,15.4048
32,0.0058,0.0059,0.0060,16.3302
64,0.0052,0.0059,0.0059,19.2016
128,0.0051,0.0057,0.0057,31.8497
256,0.0051,0.0060,0.0060,114.3355
512,0.0052,0.0057,0.0057,406.1118


In [198]:
final_results[final_results.window=='ts'].assign(layers=lambda df: df.layers.astype(int)).set_index(['model', 'window', 'batchsize', 'layers', 'l2', 'dropout'])[['loss', 'val_loss', 'test_eval', 'timing']].sort_index().style.background_gradient().set_precision(3)

# Collect batch size results

In [206]:
import glob
filenames = glob.glob(root_path + "/*seq_*batchsize.*.pickle")
filenames.sort()
batch_results = collect_results(filenames)
batch_results = batch_results.groupby('name').last().reset_index()
batch_results = process_name(batch_results)

In [224]:
batch_results[(batch_results.window=='ts') & (batch_results.dropout=='False')].assign(layers=lambda df: df.layers.astype(np.int)).assign(batchsize=lambda df: df.batchsize.astype(np.int)).set_index(['model', 'window', 'layers', 'l2', 'dropout', 'batchsize']).sort_index()[['loss', 'val_loss', 'test_eval']].style.background_gradient().set_precision(3)

# Collect feature selection results numbers

In [232]:
import glob
filenames = glob.glob(root_path + "/*feature*.pickle")
filenames.sort()
selection_results = collect_results(filenames)
selection_results = selection_results.groupby('name').last().reset_index()
selection_results = process_name(selection_results)

In [242]:
selection_results[(selection_results.l2 == 'True') & (selection_results.dropout=='False') & (selection_results.window=='tsshift')].assign(layers=lambda df: df.layers.astype(np.int)).set_index(['model', 'batchsize', 'layers', 'l2', 'dropout'])[['loss', 'val_loss', 'test_eval']].style.background_gradient().set_precision(3)